# 야구기록

In [6]:
import pandas as pd

url = 'https://sports.news.naver.com/kbaseball/record/index?category=kbo&year=2017'
df = pd.read_html(url)
df[2]

,타율 순위,타점 순위,홈런 순위,도루 순위
0,선수 페이지 1 김선빈 KIA 0.370 2 박건우 두산 0.366 ...,선수 페이지 1 러프 삼성 124 2 최형우 KIA 120 3 김...,선수 페이지 1 최정 SK 46 2 로사리오 한화 37 3 김재환...,선수 페이지 1 박해민 삼성 40 2 버나디나 KIA 32 3 손...


In [2]:
import os
import sys
import urllib.request
import datetime
import time
import json
import urllib.parse

# 네이버 API 인증 정보
client_id = 'CQxzhjUklZkXcffefOEm'
client_secret = 'aqX2vdX19L'

# [CODE 1] - 요청 함수
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print(f"[{datetime.datetime.now()}] URL 요청 성공")
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print(f"[{datetime.datetime.now()}] URL 요청 실패")
        return None

# [CODE 2] - 검색 함수
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = f"/{node}.json"
    parameters = f"?query={urllib.parse.quote(srcText)}&start={start}&display={display}"
    url = base + node + parameters

    responseDecode = getRequestUrl(url)
    if responseDecode is None:
        return None
    else:
        return json.loads(responseDecode)

# [CODE 3] - 데이터 가공
def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    jsonResult.append({
        'cnt': cnt,
        'title': title,
        'description': description,
        'org_link': org_link,
        'link': link,
        'pDate': pDate
    })

# [CODE 4] - main 함수
def main():
    node = 'news'
    srcText = input("검색어를 입력하세요: ")
    cnt = 0
    jsonResult = []

    jsonResponse = getNaverSearch(node, srcText, 1, 100)
    total = jsonResponse['total']

    while (jsonResponse is not None) and (jsonResponse['display'] != 0):
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt)

        start = jsonResponse['start'] + jsonResponse['display']
        if start > total:
            break
        jsonResponse = getNaverSearch(node, srcText, start, 100)

    print(f"전체 검색 결과 수: {total} 건")

    with open(f"{srcText}_naver_news.json", 'w', encoding='utf8') as outfile:
        json.dump(jsonResult, outfile, indent=4, sort_keys=True, ensure_ascii=False)

    print(f"{srcText}_naver_news.json SAVED")

if __name__ == '__main__':
    main()

[2025-04-03 15:59:50.754835] URL 요청 성공
[2025-04-03 15:59:51.051686] URL 요청 성공
[2025-04-03 15:59:51.340280] URL 요청 성공
[2025-04-03 15:59:51.683890] URL 요청 성공
[2025-04-03 15:59:52.208589] URL 요청 성공
[2025-04-03 15:59:52.527729] URL 요청 성공
[2025-04-03 15:59:52.872983] URL 요청 성공
[2025-04-03 15:59:53.213186] URL 요청 성공
[2025-04-03 15:59:53.509367] URL 요청 성공
[2025-04-03 15:59:53.969278] URL 요청 성공
HTTP Error 400: Bad Request
[2025-04-03 15:59:54.182289] URL 요청 실패
전체 검색 결과 수: 5165808 건
야구_naver_news.json SAVED
